In [1]:
using QuantumOptics

In [2]:
b = SpinBasis(1//2)
sx = sigmax(b)
sy = sigmay(b)
sz = sigmaz(b)

Operator(dim=2x2)
  basis: Spin(1/2)sparse([1, 2], [1, 2], ComplexF64[1.0 + 0.0im, -1.0 + 0.0im], 2, 2)

In [3]:
factor = 1
Bmag = 1

1

In [4]:
function B0(t)
    return Bmag
end

function θ(t)
    return pi/factor
end

function ϕ(t)
    return t
end

function Bx(t)
    return B0(t)*sin(θ(t))*cos(ϕ(t))
end

function By(t)
    return B0(t)*sin(θ(t))*sin(ϕ(t))
end

function Bz(t)
    return B0(t)*cos(θ(t))
end

Bz (generic function with 1 method)

In [5]:
Ht = TimeDependentSum(Bx=>sx, By=>sy, Bz=>sz)

UndefVarError: UndefVarError: `TimeDependentSum` not defined

In [6]:
function H_t(t, psi)
    return Bx(t)*sx+By(t)*sy+Bz(t)*sz
end

H_t (generic function with 1 method)

In [16]:
t=0
# E0, Psi0 = eigenstates(dense(Bx(t)*sx+By(t)*sy+Bz(t)*sz));
E0, Psi0 = eigenstates(dense(H_t(t, 0)));
E0

2-element Vector{Float64}:
 -1.0
  1.0

In [9]:
ψ₀ = Psi0[1]
dt = 0.001
tspan = [0:dt:2*pi+dt;];

UndefVarError: UndefVarError: `Psi0` not defined

In [10]:
ψ₀

UndefVarError: UndefVarError: `ψ₀` not defined

In [11]:
@time begin
tout, ψₜ = timeevolution.schroedinger_dynamic(tspan, ψ₀, H_t);
end;

UndefVarError: UndefVarError: `tspan` not defined

In [219]:
H = LazySum([0.0, 0.0, 0.0],[sx,sy,sz])
function H_B(t, psi)
  H.factors[1] = Bx(t)
  H.factors[2] = By(t)
  H.factors[3] = Bz(t)
  return H
end

H_B (generic function with 1 method)

In [220]:
@time begin
tout2, ψₜ2 = timeevolution.schroedinger_dynamic(tspan, ψ₀, H_B);
end;

  0.045322 seconds (25.61 k allocations: 1.942 MiB, 94.59% compilation time: 100% of which was recompilation)


In [221]:
t=0 
eigenenergies(dense(Bx(t)*sx+By(t)*sy+Bz(t)*sz))

2-element Vector{Float64}:
 -1.0
  1.0

In [222]:
Elist = Float64[]
for t in tspan
    #Evals = eigenenergies(dense(Bx(t)*sx+By(t)*sy+Bz(t)*sz))
    Evals = eigenenergies(dense(H_t(t)))
    push!(Elist,Evals[1])
end

In [223]:
Elist

6285-element Vector{Float64}:
 -1.0
 -0.9999999999999996
 -1.0
 -1.0
 -1.0000000000000004
 -1.0
 -1.0000000000000009
 -1.0000000000000004
 -1.0
 -1.0
  ⋮
 -0.9999999999999996
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0000000000000004
 -1.0

In [224]:
#import Pkg
#Pkg.add("NumericalIntegration")

In [225]:
using NumericalIntegration

In [226]:
# setup some data
x = collect(-π/2 : π/1000 : π/2)
y = sin.(x)

# integrate using the default Trapezoidal method
integrate(x, y)

-3.638582490861353e-16

In [227]:
Elist

6285-element Vector{Float64}:
 -1.0
 -0.9999999999999996
 -1.0
 -1.0
 -1.0000000000000004
 -1.0
 -1.0000000000000009
 -1.0000000000000004
 -1.0
 -1.0
  ⋮
 -0.9999999999999996
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0000000000000004
 -1.0

In [228]:
dynamic_phase = - NumericalIntegration.integrate(tspan,Elist)
dynamic_phase/(2*pi), angle( dagger(ψₜ[end])*ψ₀ )/pi, angle( dagger(ψₜ2[end])*ψ₀ )/pi

(1.0001296623894702, -0.000259478999648204, -0.000259478999648204)

In [229]:
solid_angle = 2*pi*(1-cos(pi/factor))

12.566370614359172

In [230]:
mod( angle( dagger(ψ₀)*ψₜ[end] ) - dynamic_phase, 2*pi)/pi, solid_angle/2/pi

(1.5422070762142147e-7, 2.0)